In [23]:
import pandas as pd
import numpy as np
from  graphviz import Source
from scipy import misc
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, precision_score, recall_score, hamming_loss
import matplotlib.pyplot as plt
import seaborn as sns

In [24]:
dataframe = pd.DataFrame()
for x in range(1,11):
    dataframe_name = 'data/block_'+str(x)+'.csv'
    new_dataframe = pd.read_csv(dataframe_name)
    dataframe = pd.concat([dataframe,new_dataframe])

frame = dataframe

In [27]:
broken_columns = list(frame.columns[2:11])

for column in broken_columns:
    frame[column] = frame[column].apply(lambda x: np.NaN if x == '?' else x)
    frame[column] = frame[column].apply(lambda x: float(x) if type(x) == str else x)

In [30]:
frame.drop(['cmp_fname_c2','cmp_lname_c2'],axis=1,inplace=True)
frame.head()

,cmp_fname_c1,cmp_lname_c1,cmp_sex,cmp_bd,cmp_bm,cmp_by,cmp_plz,is_match
0,0.833333,1.0,1,1.0,1.0,1.0,0.0,True
1,1.000000,1.0,1,1.0,1.0,1.0,1.0,True
2,1.000000,1.0,1,1.0,1.0,1.0,1.0,True
3,1.000000,1.0,1,1.0,1.0,1.0,1.0,True
4,1.000000,1.0,1,1.0,1.0,1.0,1.0,True


In [31]:
def preparer_data(frame):
    frame["cmp_fname_c1"] = frame["cmp_fname_c1"].replace(np.NaN,0.000235404896421846)
    frame["cmp_lname_c1"] = frame["cmp_lname_c1"].replace(np.NaN,2.68694413843136e-05)
    frame["cmp_sex"] = frame["cmp_sex"].replace(np.NaN,0.5)
    frame["cmp_bd"] = frame["cmp_bd"].replace(np.NaN,0.032258064516129)
    frame["cmp_bm"] = frame["cmp_bm"].replace(np.NaN,0.0833333333333333)
    frame["cmp_by"] = frame["cmp_by"].replace(np.NaN, 0.00943396226415094)
    frame["cmp_plz"] = frame["cmp_plz"].replace(np.NaN, 0.000422654268808115)
    
    return frame

frame = preparer_data(frame)


#### Criação da coluna de alvos
Para conseguir classificar os dados de acordo com coluna de alvos,
onde o sklearn index valores numericos aos dados categoricos que deseja-se
obter a predição.

In [32]:
def make_target_frame(frame, target_column):
    df_mod = frame.copy()
    targets = df_mod[target_column].unique()
    map_to_int = {name: n for n, name in enumerate(targets)}
    df_mod["Target"] = df_mod[target_column].map({True:1, False:0})
    return (df_mod, targets)
df2, targets = make_target_frame(frame, "is_match")


In [33]:
df2[["Target", "is_match"]].head()


,Target,is_match
0,1,True
1,1,True
2,1,True
3,1,True
4,1,True


In [34]:
df2[["Target", "is_match"]].tail()

,Target,is_match
574908,0,False
574909,0,False
574910,0,False
574911,0,False
574912,0,False


Partindo da tabela de correlação podemos intenficar, que os dados estão bem desacoplados, sendo assim podemos utilizalos no nosso modelo.

In [38]:
frame.head()

,cmp_fname_c1,cmp_lname_c1,cmp_sex,cmp_bd,cmp_bm,cmp_by,cmp_plz,is_match
0,0.833333,1.0,1,1.0,1.0,1.0,0.0,True
1,1.000000,1.0,1,1.0,1.0,1.0,1.0,True
2,1.000000,1.0,1,1.0,1.0,1.0,1.0,True
3,1.000000,1.0,1,1.0,1.0,1.0,1.0,True
4,1.000000,1.0,1,1.0,1.0,1.0,1.0,True


#### Criação da visualização da árvore de decisão 
Seleciona-se as colunas do frame que farão parte da analise, para esse problema
pela comando da questão as duas primeiras colunas não iriam compor a analise,
após inicializa-se o treinamento do modelo com as colunas selecionadas em x e y,
prepara-se a visualização da árvore.

In [41]:
columns = list(df2.columns[0:7])
y = df2["Target"]
x = frame[columns]
tree = DecisionTreeClassifier()
tree.fit(x,y)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [18]:


def visualize_tree(tree, feature_names):
    
    with open("tree.dot", 'w') as f:
        export_graphviz(tree, out_file=f,
                        feature_names=feature_names)

    command = ["dot", "-Tpng", "tree.dot", "-o", "tree.png"]
    try:
        subprocess.check_call(command)
    except:
        exit("Não foi possivel abrir o arquivo")
    
visualize_tree(tree, columns)

o resultado da visualização da arvore de decisão é a seguinte:

![DecisionTree](tree.png "DecisionTree")

# Avaliando a precisão do modelo
Para avaliar a precisão do modelo os grupos de dados são divididos em dados de treino e dados de teste, onde os dados de treino são x_train e y_train, e os dados de teste são, x_test e y_test. Para realizar a separação de dados utiliza-se o método train_test_split(), da seguinte forma:

In [42]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.33, random_state=42)

Após dividir os grupos de dados, sendo que os dados de teste são definidos em 33% e os dados de treino são 67% dos dados originais, então é realizado o treino dos dados utilizando o método fit(), a previsão dos dados de teste utilizando o método predict() e uma matriz de confusão dos dados de teste com o método confusion_matrix(), como em:

In [43]:
tree.fit(x_train, y_train)
preds = tree.predict(x_test)

Com os dados treinados e previstos, gera-se a matriz de confusão onde existem os dados true_negative(tn), false_positive(fp), false_negative(fn) e true_positive(tp):

In [44]:
tn, fp, fn, tp = confusion_matrix(y_test, preds).ravel()
(tn, fp, fn, tp)

(1890327, 10, 13, 6864)

Após a matriz de confusão, para a avaliação do modelo realiza-se a comparação dos dados de teste previstos e os dados classificados escolhidos aleatoriamente utilizando o método accuracy_score() e obtém-se a precisão da classificação:

<b>tn</b> = Representa a quantidade de *is_match* = false, que foram acertados corretamente pelo modelo.<br/>
<b>fp</b> = Representa a quantidade de *is_match* = true, que foram errados pelo modelo.<br/>
<b>fn</b> = Representa a quantidade de *is_match* = false, que foram errados pelo modelo.<br/>
<b>tp</b> = Representa a quantidade de *is_match* = true, que foram acertados corretamente pelo modelo.<br/>

#### Acuracia

Afim de saber a capacidade do sistema de concluir de forma correta a classificação dos dados.

In [45]:
print(accuracy_score(y_test, preds))

0.9999878769606381


O valor da precisão obtido é de 99,9% o que sugere que em 99,9% das vezes a árvore de decisão é capaz de prever se o "match" é verdadeiro ou falso. 

#### Precisão

A precisão é a capacidade que o modelo tem em não rotular uma amostra que é classificada como positiva em negativa,
abaixo podemos observar a capacidade do modelo em expressar isso.

In [46]:
print(precision_score(y_test, preds))

0.9985452429444283


O valor obtido da precisão é de 99.85%, o que sugere que em 99.85% das vezes o modelo rotula corretamente uma amostra positiva.

#### Recall

recall é a capacidade do modelo de encontrar todos os valores positivos na base

In [47]:
print(recall_score(y_test, preds))

0.998109640831758


É possível observar que o recall atestado é de 99,81%, significa que o modelo é capaz de encontrar quase todos os dados positivos do sistema 

#### Perda de Hamming
é a fração da qual o modelo prediz incorretamente.

In [48]:
print(hamming_loss(y_test, preds))

1.2123039361927541e-05


Percebe-se que em aproximadamente 0.0012% das vezes a árvore de decisão realiza previsões incorretas.